# Your first RAG application

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import rich

import ollama 

In [2]:
# Obtain your embedding & LLM models (execute here or go to terminal)
!ollama pull mxbai-embed-large
!ollama pull mistral  

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 819c2adf5ce6: 100% ▕██████████████████▏ 669 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling b837481ff855: 100% ▕██████████████████▏   16 B                         
pulling 38badd946f91: 100% ▕██████████████████▏  408 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling ff82381e2bea: 100% ▕██████████████████▏ 4.1 GB                         
pulling 43070e2d4e53: 100% ▕██████████████████▏  11 KB                         
pulling 491dfa501e59: 100% ▕██████████████████▏  801 B                         
pulling ed11eda7790d: 100% ▕██████████████████▏   30 B                         
pulling 42347cd80dc8: 100% ▕██████████████████▏  485 B                         
ve

In [3]:
# Check model availability 
!ollama list

NAME                        ID              SIZE      MODIFIED               
mistral:latest              f974a74358d6    4.1 GB    Less than a second ago    
mxbai-embed-large:latest    468836162de7    669 MB    Less than a second ago    


In [4]:
# First run 
res = ollama.chat(model="mistral", 
            messages=[ {"role": "user", "content": "Tell me a joke about Data Science"}]
           )
rich.print(res)

ChatResponse(
    model='mistral',
    created_at='2025-05-09T21:02:08.839524Z',
    done=True,
    done_reason='stop',
    total_duration=5827962542,
    load_duration=16437000,
    prompt_eval_count=12,
    prompt_eval_duration=466928375,
    eval_count=64,
    eval_duration=5343878125,
    message=Message(
        role='assistant',
        content=' Why don\'t Data Scientists go out in the rain?\n\nBecause they have too many features and only 
one label for "wet"!\n\n(But in all seriousness, Data Scientists love working with large datasets and finding 
patterns, even if it means getting drenched in data!)',
        images=None,
        tool_calls=None
    )
)

## Set up your first RAG pipeline

The main components of a naive RAG pipeline are: 

1. Index your documents (embed) 
2. Build a retriever
3. Augment the capabilities of the LLM 
4. Synthesis of the answer 

In [5]:
import os
import ollama
import chromadb
from chromadb.utils.embedding_functions import DefaultEmbeddingFunction

In [6]:
# === Step 1: Setup ChromaDB ===
chroma_client = chromadb.Client()
collection    = chroma_client.get_or_create_collection(name="rag-docs")

In [7]:
# === Step 2: Load and Embed Documents ===
def embed_text(text):
    response = ollama.embed(model="mxbai-embed-large", input=text)
    return response["embeddings"][0]

In [8]:
# Sample docs (could also read from files)
documents = [
    "Jurgen Klopp was born in Germany in 1974. He has been a successful coach in the UK",
    "You can contact Sky customer support through the help portal or live chat.",
    "An apple a day keeps the doctor away"
]

In [9]:
for i, doc in tqdm(enumerate(documents)):
    embedding = embed_text(doc)
    collection.add( documents=[doc],
                    embeddings=[embedding],
                    ids=[f"doc-{i}"]
                    )

0it [00:00, ?it/s]

In [10]:
# === Step 3: Accept User Query and Retrieve Relevant Docs ===
query = "What team did Jurgen Klopp coach?"

query_embedding = embed_text(query)
results = collection.query(query_embeddings=[query_embedding], n_results=3)

rich.print(results) 

{
    'ids': [['doc-0', 'doc-1', 'doc-2']],
    'embeddings': None,
    'documents': [
        [
            'Jurgen Klopp was born in Germany in 1974. He has been a successful coach in the UK',
            'You can contact Sky customer support through the help portal or live chat.',
            'An apple a day keeps the doctor away'
        ]
    ],
    'uris': None,
    'included': ['metadatas', 'documents', 'distances'],
    'data': None,
    'metadatas': [[None, None, None]],
    'distances': [[0.3585015535354614, 1.367884874343872, 1.476802945137024]]
}

In [11]:
retrieved_docs = results["documents"][0]
context = "\n".join(retrieved_docs)

In [12]:
# === Step 4: Run RAG Prompt through Ollama LLM ===
answer_prompt = """You're a personal assistant. Your task is to answer questions using only the provided context. 
If you can not explicitly extract the answer from the context, your answer must be I cannot help with that. 

Context: {context}

Question: {query}

Answer:"""

rich.print(answer_prompt)

You're a personal assistant. Your task is to answer questions using only the provided context. 
If you can not explicitly extract the answer from the context, your answer must be I cannot help with that. 

Context: {context}

Question: {query}

Answer:

In [13]:
query = 'What team did Jurgen Klopp coach?'

In [14]:
res = ollama.chat(model="mistral", 
            messages=[ {"role": "user", "content": answer_prompt.format(context=context, query=query)}]
           )
rich.print(res)

ChatResponse(
    model='mistral',
    created_at='2025-05-09T21:02:13.913544Z',
    done=True,
    done_reason='stop',
    total_duration=3809865000,
    load_duration=6716375,
    prompt_eval_count=119,
    prompt_eval_duration=1555983417,
    eval_count=26,
    eval_duration=2246663250,
    message=Message(
        role='assistant',
        content=' I cannot help with that, as the provided context does not mention which team Jurgen Klopp coached
in the UK.',
        images=None,
        tool_calls=None
    )
)

## Build a proper index 

1. Take large documents and chunk them if needed
2. Add relevant metadata to the documents to enhance search
3. Embed and add to the collection 

In [15]:
# 40,000 lines of Shakespeare from a variety of Shakespeare's plays
with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text = text[:100000]

In [16]:
# Naive splitter 
def chunk_text(text, chunk_size, overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap  # move back by `overlap` characters
    return chunks

chunks = chunk_text( text, chunk_size=2000, overlap=200 )
print( f'The split method produced {len(chunks)} chunks' )

The split method produced 56 chunks


In [17]:
shakespeare_collection = chroma_client.get_or_create_collection(name="shakespeare-chunks")

i=0
for chunk in tqdm(chunks): 
    embedding = embed_text(chunk)
    shakespeare_collection.add( documents=[chunk],
                    embeddings=[embedding],
                    ids=[f"{i}"]
                    )
    i+=1

  0%|          | 0/56 [00:00<?, ?it/s]

## Generate a valid set of questions so I can evaluate the Retrieval & Generation

Idea is simple: by extracting the questions from passages:
- I can keep track of the origin 
- I can set up an evaluation method for retrieval efficacy. 

In [18]:
summary_prompt = """You are an expert Shakespeare analyst. You will receive a chunk of one of his books, and your task is to summarise what is happening in the passage.
Write a short summary capturing the most relevant information of the passage in less than 100 words. 

Chunk: {chunks}

Answer: 
"""

question_prompt = """You are an expert Shakespeare analyst. You will receive a summary of a passage of one of his books. 
Your task is to generate ONE simple, short, fact-based question that can be answered with the provided text alone. 

You must not mention that you are extracting the question from a text, a passage or a chunk.

Text: {summary}

Question: 
"""

rich.print(f' "SUMMARY_PROMPT" = {summary_prompt}')
rich.print('------------')
rich.print(f' "QUESTION_PROMPT" = {question_prompt}')

"SUMMARY_PROMPT" = You are an expert Shakespeare analyst. You will receive a chunk of one of his books, and your 
task is to summarise what is happening in the passage.
Write a short summary capturing the most relevant information of the passage in less than 100 words. 

Chunk: {chunks}

Answer:

------------

"QUESTION_PROMPT" = You are an expert Shakespeare analyst. You will receive a summary of a passage of one of his 
books. 
Your task is to generate ONE simple, short, fact-based question that can be answered with the provided text alone. 

You must not mention that you are extracting the question from a text, a passage or a chunk. Avoid 
"in this passage from xxx", references to the name of the book, or similar self-references.

Text: {summary}

Question:

In [19]:
def extract_question(chunks, chunk_id, verbose=False): 

    # Extract a summary from the provided passage
    summary = ollama.chat(model="mistral", messages=[
    {"role": "user", "content": summary_prompt.format(chunks=chunks[chunk_id])}
    ])

    if verbose:
        rich.print( f'Summary from chunk {chunk_id}: {summary["message"]["content"]}')

    # Extract question from the generated summary 
    question = ollama.chat(model="mistral", messages=[
    {"role": "user", "content": question_prompt.format(summary=summary["message"]["content"])}
    ])

    question = question["message"]["content"]
    rich.print( f'Question from chunk {chunk_id}: {question}')

    return question 

In [20]:
question = extract_question( chunks, 1, verbose=True )

Summary from chunk 1:  The passage is from Coriolanus Act 1 Scene 1 and presents a dispute between the patricians 
(nobles) and plebeians (commoners) in Rome. The people are angry due to a food shortage, believing that the nobles 
are hoarding grain while they suffer. They have armed themselves and are marching towards the Capitol, intending to
show their discontent with deeds, not just words. Menenius Agrippa, a noble who is well-liked by the people, tries 
to pacify them, stating that the nobles care for them like fathers. However, the first citizen expresses bitterness
towards the nobles, accusing them of not caring for their welfare and making laws to oppress the poor. The scene 
ends with Menenius threatening to accuse the people if they do not change their actions. This tension between the 
classes sets the stage for further conflict throughout the play.

Question from chunk 1:  What is the primary cause of tension between the patricians and plebeians in Coriolanus as 
shown at the beginning of the play?

In [ ]:
chunk_ids = [1,20,35,42,48]
questions = []

for chunk_id in tqdm(chunk_ids):
    question = extract_question( chunks, chunk_id )
    questions.append(question)

question_set = pd.DataFrame( {'chunk_id':chunk_ids, 'question': questions} )

  0%|          | 0/5 [00:00<?, ?it/s]

Question from chunk 1:  Who do the citizens in Coriolanus accuse of exploitation and indifference?

Question from chunk 20:  Who does Menenius accuse of hypocrisy in this dialogue?

Question from chunk 35:  Who does Brutus advise against electing as a consul in "The Tragedy of Julius Caesar"?

Question from chunk 42:  Who orders the guards to seize Coriolanus in this passage?

# Retrieval 

In [ ]:
k = 20 

def find_position(lst, value):
    try:
        return lst.index(value)
    except ValueError:
        return np.nan

In [ ]:
found = []
retrieved_chunks = []
for idx in tqdm(question_set.index):
    q   = question_set.at[idx, 'question'] 
    cid = question_set.at[idx, 'chunk_id'] 

    query_embedding = embed_text(q)
    results         = shakespeare_collection.query(query_embeddings=[query_embedding], n_results=k)

    retrieved_chunks.append( results['ids'][0] )
    found.append( find_position( results['ids'][0], str(cid) ) )

question_set['retrieval_rank']   = found 
question_set['retrieved_chunks'] = retrieved_chunks

In [ ]:
# Retrieval results 
question_set

In [ ]:
import matplotlib.pyplot as plt

ranks     = range(0,k)
precision = [ (question_set['retrieval_rank']<=rk).mean() for rk in ranks ]

plt.title( 'Precision @k' )
plt.plot( ranks, precision )
plt.xlabel( 'k' )
plt.ylabel( 'Precision' )

# Augment & Generate

In [ ]:
top_k     = 5
synthesis = []

for idx in tqdm(question_set.index):

    question = question_set.at[idx, 'question']
    context  = "\n".join( [ chunks[int(cid)] for cid in question_set.at[idx, 'retrieved_chunks'][:top_k] ] )

    response = ollama.chat(model="mistral", messages=[
    {"role": "user", "content": answer_prompt.format( context=context, query=question ) }
])
    answer = response["message"]["content"]
    rich.print( f' "Query": {question} \n "Answer": {answer}' )
    synthesis.append( answer )

question_set['Responses'] = synthesis 